In [0]:
dim_atleta = 'workspace.project_data_football_silver.dim_atleta'
dim_campeonato = 'workspace.project_data_football_silver.dim_campeonato'
dim_clube = 'workspace.project_data_football_silver.dim_clube'
dim_estadio = 'workspace.project_data_football_silver.dim_estadio'
dim_posicao = 'workspace.project_data_football_silver.dim_posicao'
dim_rodada = 'workspace.project_data_football_silver.dim_rodada'
fato_partida = 'workspace.project_data_football_silver.fato_partida'
fato_pontuacao = 'workspace.project_data_football_silver.fato_pontuacao'

In [0]:
spark.table(fato_pontuacao).createOrReplaceTempView("tmp_silver_pontuacao")
spark.table(dim_clube).createOrReplaceTempView("tmp_silver_clube")
spark.table(dim_atleta).createOrReplaceTempView("tmp_silver_atleta")
spark.table(dim_posicao).createOrReplaceTempView("tmp_silver_posicao")
spark.table(dim_campeonato).createOrReplaceTempView("tmp_silver_campeonato")

In [0]:
# Consulta SQL para calcular KPIs de pontuação por rodada, posição e clube
df_gold_kpi_pontuacoes_cartola = spark.sql("""
  WITH base AS (
    -- Base de pontuações com informações de atleta, clube, posição e campeonato
    SELECT
      p.campeonato_id,
      c.nome AS campeonato_nome,
      c.temporada,
      p.rodada,
      p.atleta_id,
      a.nome AS atleta_nome,
      p.clube_id,
      cl.nome_completo AS clube_nome,
      pos.nome AS posicao_nome,
      p.pontuacao
    FROM
      tmp_silver_pontuacao p
        JOIN tmp_silver_atleta a ON p.atleta_id = a.atleta_id
        JOIN tmp_silver_clube cl ON p.clube_id = cl.clube_id
        JOIN tmp_silver_posicao pos ON a.posicao_id = pos.posicao_id
        JOIN tmp_silver_campeonato c ON p.campeonato_id = c.campeonato_id
  ),
  media_posicao AS (
    -- Média de pontuação por posição
    SELECT
      campeonato_id,
      campeonato_nome,
      temporada,
      posicao_nome,
      ROUND(AVG(pontuacao), 2) AS media_pontuacao
    FROM base
    GROUP BY campeonato_id, campeonato_nome, temporada, posicao_nome
  ),
  media_clube AS (
    -- Média de pontuação por clube
    SELECT
      campeonato_id,
      campeonato_nome,
      temporada,
      clube_nome,
      ROUND(AVG(pontuacao), 2) AS media_pontuacao
    FROM base
    GROUP BY campeonato_id, campeonato_nome, temporada, clube_nome
  )
  -- KPIs: atletas mais/menos pontuados, posição e clubes com maior/menor média
  SELECT
    campeonato_nome,
    temporada,
    rodada,
    'Atleta mais pontuado da rodada' AS indicador,
    CONCAT_WS(', ', SORT_ARRAY(COLLECT_SET(atleta_nome))) AS referencia,
    MAX(pontuacao) AS valor
  FROM base b
  WHERE pontuacao = (
    SELECT MAX(bb.pontuacao)
    FROM base bb
    WHERE bb.rodada = b.rodada AND bb.campeonato_id = b.campeonato_id
  )
  GROUP BY campeonato_nome, temporada, rodada
  UNION ALL
  SELECT
    campeonato_nome,
    temporada,
    rodada,
    'Atleta menos pontuado da rodada',
    CONCAT_WS(', ', SORT_ARRAY(COLLECT_SET(atleta_nome))),
    MIN(pontuacao)
  FROM base b
  WHERE pontuacao = (
    SELECT MIN(bb.pontuacao)
    FROM base bb
    WHERE bb.rodada = b.rodada AND bb.campeonato_id = b.campeonato_id
  )
  GROUP BY campeonato_nome, temporada, rodada
  UNION ALL
  SELECT
    campeonato_nome,
    temporada,
    NULL AS rodada,
    'Posição com maior média',
    CONCAT_WS(', ', SORT_ARRAY(COLLECT_SET(posicao_nome))),
    MAX(media_pontuacao)
  FROM media_posicao mp
  WHERE media_pontuacao = (
    SELECT MAX(media_pontuacao)
    FROM media_posicao x
    WHERE x.campeonato_id = mp.campeonato_id
  )
  GROUP BY campeonato_nome, temporada
  UNION ALL
  SELECT
    campeonato_nome,
    temporada,
    NULL,
    'Clube com maior média de pontuação',
    CONCAT_WS(', ', SORT_ARRAY(COLLECT_SET(clube_nome))),
    MAX(media_pontuacao)
  FROM media_clube mc
  WHERE media_pontuacao = (
    SELECT MAX(media_pontuacao)
    FROM media_clube x
    WHERE x.campeonato_id = mc.campeonato_id
  )
  GROUP BY campeonato_nome, temporada
  UNION ALL
  SELECT
    campeonato_nome,
    temporada,
    NULL,
    'Clube com menor média de pontuação',
    CONCAT_WS(', ', SORT_ARRAY(COLLECT_SET(clube_nome))),
    MIN(media_pontuacao)
  FROM media_clube mc
  WHERE media_pontuacao = (
    SELECT MIN(media_pontuacao)
    FROM media_clube x
    WHERE x.campeonato_id = mc.campeonato_id
  )
  GROUP BY campeonato_nome, temporada
  ORDER BY campeonato_nome, rodada, indicador;
""")

# Salva o resultado como tabela Delta no layer gold
df_gold_kpi_pontuacoes_cartola.write\
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("workspace.project_data_football_gold.kpi_pontuacoes_cartola")